Imports

In [1]:
import os
import sys
import math
import numpy as np
from PIL import Image

### Helper functions

Function to normalize a 2d array

In [2]:
def GetMaxValue(image_array):
    x, y = image_array.shape
    max_value = 0
    for i in range(x):
        for j in range(y):
            value = image_array[i][j]
            if value > max_value:
                max_value = value
    return max_value

def NormalizeArray(image_array):
    x, y = image_array.shape
    max_value = GetMaxValue(image_array)
    normalized_image_array = np.copy(image_array)
    for i in range(x):
        for j in range(y):
            old_value = image_array[i][j]
            new_value = round(old_value / max_value * 255)
            normalized_image_array[i][j] = new_value
    return normalized_image_array

Function to convert a greyscale image to 2D array

In [3]:
def ImageToArray(image):
    image_array = np.array(image.getdata(), dtype=np.uint8)
    image_array = image_array.reshape(image.size)
    return image_array

Function to convert a 2d array to an image

In [4]:
def ArrayToImage(image_array):
    x, y = image_array.shape
    image = Image.new("L", (x, y))
    for i in range(x):
        for j in range(y):
            value = int(image_array[i][j])
            image.putpixel((i, j), value)
    return image

### Main functions

CalculateIntegral function

In [5]:
def CalculateIntegral(image):
    x, y = image.size
    row_sums = np.zeros((x+1, y+1), dtype=np.uint32)
    integral_image = np.zeros((x+1, y+1), dtype=np.uint32)
    for i in range(x):
        for j in range(y):
            r = i+1
            c = j+1
            row_sums[r][c] = row_sums[r][c-1] + image.getpixel((i, j))
            integral_image[r][c] = integral_image[r-1][c] + row_sums[r][c]
    integral_image = np.delete(integral_image, 0, axis=0)
    integral_image = np.delete(integral_image, 0, axis=1)
    return integral_image

CalculateLocalSum function

In [6]:
def CalculateLocalSum(integral_image, p0, p1):
    x0, y0 = p0
    x1, y1 = p1
    sum_x = integral_image[x0][y0]
    sum_y = integral_image[x1][y0]
    sum_z = integral_image[x0][y1]
    sum_k = integral_image[x1][y1]
    local_sum = sum_k + sum_x - sum_z - sum_y
    return local_sum

DetectEye function

In [7]:
def CalculateKernel(integral_image, p, d):
    i, j = p
    n, m = d
    
    p1 = (i - math.floor(0.5*n), j - math.floor(0.5*m))
    p2 = (i - math.floor(0.05*n), j)
    p3 = (i - math.floor(0.5*n), j)
    p4 = (i - math.floor(0.05*n), j + math.floor(0.5*m))
    p5 = (i + math.floor(0.05*n), j - math.floor(0.5*m))
    p6 = (i + math.floor(0.5*n), j)
    p7 = (i + math.floor(0.05*n), j)
    p8 = (i + math.floor(0.5*n), j + math.floor(0.5*m))
    p9 = (i - math.floor(0.325*n), j + math.floor(0.833*m))
    p10 = (i - math.floor(0.225*n), j + 2*m)
    p11 = (i - math.floor(0.1*n), j + math.floor(0.833*m))
    p12 = (i + math.floor(0.1*n), j + 2*m)
    p13 = (i + math.floor(0.225*n), j + math.floor(0.833*m))
    p14 = (i + math.floor(0.325*n), j + 2*m)

    score = 0
    score += CalculateLocalSum(integral_image, p1, p2)
    score -= CalculateLocalSum(integral_image, p3, p4)
    score += CalculateLocalSum(integral_image, p5, p6)
    score -= CalculateLocalSum(integral_image, p7, p8)
    score -= CalculateLocalSum(integral_image, p9, p10)
    score += CalculateLocalSum(integral_image, p11, p12)
    score -= CalculateLocalSum(integral_image, p13, p14)
    
    return score

In [8]:
def DetectEye(integral_image, n):
    x, y = integral_image.shape
    m = round(0.15 * n)
    if n % 2 == 0:
        n -= 1
    if math.ceil(m) % 2 != 0:
        m = math.ceil(m)
    else:
        m = math.floor(m)
    
    i_start = math.floor(n / 2)
    i_end = x - math.floor(n / 2)
    j_start = math.floor(m / 2)
    j_end = y - math.floor(2.5 * m)
    max_score = -sys.maxsize
    max_score_point = (0, 0)
    for i in range(i_start, i_end):
        for j in range(j_start, j_end):
            score = CalculateKernel(integral_image, (i, j), (n, m))
            if score > max_score:
                max_score = score
                max_score_point = (i, j)
    return max_score_point

ExtractDetectedEye function

In [9]:
def ExtractDetectedEye(image, p_max, n):
    x_max, y_max = p_max
    m = round(0.15 * n)
    if n % 2 == 0:
        n -= 1
    if math.ceil(m) % 2 != 0:
        m = math.ceil(m)
    else:
        m = math.floor(m)
    
    i_start = x_max - math.floor(n / 2)
    i_end = x_max + math.floor(n / 2)
    j_start = y_max - math.floor(m / 2)
    j_end = y_max + math.floor(2.5 * m)
    output_image_array = np.zeros(image.size)
    for i in range(i_start, i_end):
        for j in range(j_start, j_end):
            output_image_array[i][j] = image.getpixel((i, j))
    return output_image_array

### Test

Test Part 1

In [ ]:
image_array = np.full((200, 200), 100, dtype=np.uint8)
image = ArrayToImage(image_array)

In [ ]:
integral_image = CalculateIntegral(image)
ArrayToImage(NormalizeArray(integral_image))

In [ ]:
CalculateLocalSum(integral_image, (20, 20), (30, 30))

Test Part 2

In [19]:
current_directory = os.getcwd()
image_path = current_directory + R"\images\f2.png"
image = Image.open(image_path)
image = image.convert("L")
n = 330

In [20]:
integral_image = CalculateIntegral(image)
p_max = DetectEye(integral_image, n)
detected_eye_image_array = ExtractDetectedEye(image, p_max, n)
detected_eye_image = ArrayToImage(detected_eye_image_array)
detected_eye_image.save(current_directory + R"\f2.jpg")